In [6]:
from seismic import SeismicDataset
import os
import json
import numpy as np
from tqdm import tqdm

In [16]:
import ir_datasets
import ir_measures

from ir_measures import nDCG

In [7]:
document_path = "/data1/knn_datasets/sparse_datasets/beir/beir_spladev3_fromCarlos_CROSSCHECK-MRR-BEFORE-USE/beir/quora/docs/docs_anserini.jsonl"

In [8]:
dataset = SeismicDataset()

In [9]:
MAX_TOKEN_LEN=30
string_type  = f'U{MAX_TOKEN_LEN}'

with open(document_path, "r") as file:
    for line in tqdm(file):
        line_data = json.loads(line.strip())
        vs = np.array([v for v in line_data["vector"].values()], dtype=np.float32)
        ks = np.array([k for k in line_data["vector"].keys()], dtype=string_type)
        dataset.add_document(line_data["id"], ks, vs)

       # documents.append((ks, vs))
       # doc_ids.append(line_data["id"])

522931it [00:22, 23671.29it/s]


In [10]:
from seismic import SeismicIndex

In [12]:
index = SeismicIndex.build_from_dataset(dataset)


Building the index...
Configuration { pruning: GlobalThreshold { n_postings: 3500, max_fraction: 1.5 }, blocking: RandomKmeans { centroid_fraction: 0.1, min_cluster_size: 2, clustering_algorithm: RandomKmeansInvertedIndexApprox { doc_cut: 15 } }, summarization: EnergyPreserving { summary_energy: 0.4 }, knn: KnnConfiguration { nknn: 0, knn_path: None }, batched_indexing: None }
Distributing and pruning postings: 0 secs
Number of posting lists: 26595
Building summaries: 3 secs


In [13]:
queries_path = "/data1/knn_datasets/sparse_datasets/beir/beir_spladev3_fromCarlos_CROSSCHECK-MRR-BEFORE-USE/beir/quora/queries/docs_anserini.jsonl"

queries = []
with open(queries_path, 'r') as f:
    for line in f:
        queries.append(json.loads(line))

MAX_TOKEN_LEN = 30
string_type  = f'U{MAX_TOKEN_LEN}'

queries_ids = np.ascontiguousarray(np.array([q['id'] for q in queries], dtype=string_type))

query_components = []
query_values = []

for query in queries:
    vector = query['vector']
    query_components.append(np.array(list(vector.keys()), dtype=string_type))
    query_values.append(np.array(list(vector.values()), dtype=np.float32))

In [14]:
results = index.batch_search(
    queries_ids=queries_ids,
    query_components=query_components,
    query_values=query_values,
    k=10,
    query_cut=10,
    heap_factor=0.7,
    # sorted=True,
    # n_knn=0,
)

In [21]:
# add your ir_dataset dataset string id below, e.g., "beir/quora/test"
ir_dataset_string = "beir/quora/test"

metric_name = "nDCG@10" # on BEIR/quora
ir_measure = ir_measures.parse_measure(metric_name)
ir_results = [ir_measures.ScoredDoc(query_id, doc_id, score) for r in results for (query_id, score, doc_id) in r]

qrels = ir_datasets.load(ir_dataset_string).qrels
ir_measures.calc_aggregate([ir_measure], qrels, ir_results)

ValueError: Unsupported measures {nDCG@10}. The following providers would support this measure:
 - pytrec_eval (pip install --upgrade pytrec-eval-terrier)
 - ranx (pip install ir-measures[ranx])

In [22]:
!pip install ranx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 37.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
